In [1]:
import boto3
import sagemaker
from sagemaker.estimator import Estimator

In [2]:
es = Estimator(
    role=sagemaker.get_execution_role(),
    image_uri='939595455984.dkr.ecr.eu-west-1.amazonaws.com/dgl-citation-network:custom-torch-1.8',
    instance_type='ml.m5.large',
    instance_count=1,
    hyperparameters={
        'epochs': 100,
        'learning-rate': 0.01
    }
)

In [3]:
es.fit()

2021-03-26 10:57:57 Starting - Starting the training job...
2021-03-26 10:57:59 Starting - Launching requested ML instancesProfilerReport-1616756276: InProgress
......
2021-03-26 10:59:23 Starting - Preparing the instances for training...
2021-03-26 10:59:49 Downloading - Downloading input data
2021-03-26 10:59:49 Training - Downloading the training image.........
2021-03-26 11:01:23 Training - Training image download completed. Training in progress.DGL backend not selected or invalid.  Assuming PyTorch for now.
Using backend: pytorch
/usr/local/lib/python3.9/site-packages/dgl/base.py:45: DGLWarning: DGLGraph.__len__ is deprecated.Please directly call DGLGraph.number_of_nodes.
  return warnings.warn(message, category=category, stacklevel=1)
Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)
Extracting file to /root/.dgl/cora_full
Epoch 0/

In [5]:
es_predictor = es.deploy(
    instance_type='ml.m5.large',
    initial_instance_count=1
)

-------------!

In [14]:
test_samples = ['0, 2, 4, 5']

# es_predictor.content_type = 'text/csv' 
es_predictor.serializer = sagemaker.serializers.CSVSerializer()

response = es_predictor.predict(test_samples, initial_args={'ContentType': 'text/csv'}) 
print(response)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from model with message "<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>
". See https://eu-west-1.console.aws.amazon.com/cloudwatch/home?region=eu-west-1#logEventViewer:group=/aws/sagemaker/Endpoints/dgl-citation-network-2021-03-26-11-08-36-905 in account 939595455984 for more information.

In [16]:
es_predictor.delete_endpoint()

ClientError: An error occurred (ValidationException) when calling the DeleteEndpointConfig operation: Could not find endpoint configuration "arn:aws:sagemaker:eu-west-1:939595455984:endpoint-config/dgl-citation-network-2021-03-26-11-08-36-905".